In [47]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix
from s


def specificity(y_true, y_pred, threshold=0.5):
    y_pred_thresholded = tf.cast(y_pred > threshold, tf.bool)
    y_true = tf.cast(y_true, tf.bool)

    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(y_true), tf.logical_not(y_pred_thresholded)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(y_true), y_pred_thresholded), dtype=tf.float32))

    return true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())


precison = Precision()

In [41]:
DEEP_DATA_PATH = r"C:\Users\theal\Documents\CrspData\deepData"

# path = os.path.join(DEEP_DATA_PATH, 'deepBollinger', 'scaled_combined_bollinger_naive_dynamic_sl_20100104_to_20201231_doctest.csv')
path = os.path.join(DEEP_DATA_PATH, 'deepTurtle', 'scaled_combined_turtle_naive_20100104_to_20201231_doctest.csv')
# path = os.path.join(DEEP_DATA_PATH, 'deepBox', 'scaled_combined_box_naive_20100104_to_20201231_doctest.csv')

df = pd.read_csv(path, low_memory=False)

In [42]:
trade_counts = df['is_trade'].value_counts()
print(trade_counts)

0    424456
1    420653
Name: is_trade, dtype: int64


In [43]:
# Prepare input and target
X = df[[f'PrevPrice_{i}' for i in range(21)]].values
y = df['is_trade'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [46]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precison, specificity])

NameError: name 'precison' is not defined

In [45]:
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

21128/21128 [==============================] - 104s 5ms/step - loss: 0.1631 - accuracy: 0.9358 - precision_4: 0.9121 - specificity: 0.9078 - val_loss: 0.1332 - val_accuracy: 0.9491 - val_precision_4: 0.9198 - val_specificity: 0.9146


In [16]:
results = model.evaluate(X_test, y_test)

276/276 [==============================] - 1s 3ms/step - loss: 0.2265 - accuracy: 0.9092 - precision_1: 0.8914 - specificity: 0.0000e+00


In [34]:
y_pred = model.predict(X_test)

In [38]:
y_pred = (y_pred > 0.5).astype("int32")

In [39]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[80772,  4110],
       [ 6018, 78122]], dtype=int64)